# Problem 14.02 Work and Temperature Change Upon Isentropic Compression of Oxygen

A stream of gaseous oxygen is compressed from 1 bar to 10 bar. The inlet temperature is 25 °C.
Calculate the specific work and the temperature of the outlet gas if the process as an isentropic efficiency of 1, using both the ideal gas law and the SRK equation of state.

## Solution

This requires a PT and then a PS flash only. This problem is also good for contrasting simple engineering formulas for compression vs. rigorous thermodynamics.

In [1]:
# Set the conditions and imports
from scipy.constants import bar, hour
from thermo import ChemicalConstantsPackage, SRKMIX, IdealGas, CEOSLiquid, CEOSGas, FlashPureVLS
fluid = 'oxygen'
constants, correlations = ChemicalConstantsPackage.from_IDs([fluid])

T1 = 298.15
P1 = 1*bar
P2 = 10*bar

In [2]:
# Use the Ideal-Gas EOS
gas = IdealGas(HeatCapacityGases=correlations.HeatCapacityGases)
# Note that we can set-up a flasher object with only a gas phase
# This obviously has much more performance!
flasher = FlashPureVLS(constants, correlations, gas=gas, liquids=[], solids=[])

# Flash at inlet conditions to obtain initial enthalpy
state_1 = state_1_ideal = flasher.flash(T=T1, P=P1)
# Flash at outlet condition - entropy is conserved by compressors and expanders!
state_2 = state_2_ideal = flasher.flash(S=state_1.S(), P=P2)

actual_power = (state_2.H() - state_1.H()) # W/mol
print('With the ideal-gas EOS:')
print(f'The actual power is {actual_power:.4f} J/mol')
print(f'The actual outlet temperature is {state_2.T: .2f} K')

With the ideal-gas EOS:
The actual power is 7991.3774 J/mol
The actual outlet temperature is  560.72 K


In [3]:
# SRK
eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas)
liquid = CEOSLiquid(SRKMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
gas = CEOSGas(SRKMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
flasher = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])

# Flash at inlet conditions to obtain initial enthalpy
state_1 = flasher.flash(T=T1, P=P1)
# Flash at outlet condition - entropy is conserved by compressors and expanders!
state_2 = state_2_ideal = flasher.flash(S=state_1.S(), P=P2)

actual_power = (state_2.H() - state_1.H()) # W/mol
print('With the SRK EOS:')
print(f'The actual power is {actual_power:.4f} J/mol')
print(f'The actual outlet temperature is {state_2.T: .2f} K')

With the SRK EOS:
The actual power is 8000.3185 J/mol
The actual outlet temperature is  561.08 K


These calculations make use of the full power of the Thermo engine. It is also possible to use simpler calculations to calculate, as shown below.

In [4]:
from fluids import isentropic_work_compression, isentropic_T_rise_compression
k = state_1_ideal.isentropic_exponent()
Z = state_1_ideal.Z()
print(f'Using the ideal isentropic exponent {k:.3f}')
print(f'Using the ideal compressibility {Z:.3f}')
molar_work = isentropic_work_compression(T1=T1, k=state_1_ideal.isentropic_exponent(), Z=state_1_ideal.Z(), P1=P1, P2=P2, eta=1)
T2 = isentropic_T_rise_compression(T1=T1, P1=P1, P2=P2, k=k, eta=1)
print(f'The simple power is {molar_work:.4f} J/mol')
print(f'The simple outlet temperature is {T2: .2f} K')

Using the ideal isentropic exponent 1.395
Using the ideal compressibility 1.000
The simple power is 8047.5136 J/mol
The simple outlet temperature is  572.10 K


From these results, we can see that for small pressure increases, the ideal-gas and SRK equations work quite similarly. There is also a very large difference in outlet temperature between the simplified equations given in many textbooks, and the real isentropic calculations when a temperature-dependent heat capacity is used. Therefore, there are substantial advantages to rigorous modeling, regardless of the complexity of the EOS for the gas phase.